In [52]:
host = "http://130.237.3.107:8080/api/"
api_token = "39cd216c44fc16542a706820c4d0bd4377f1a416"
input_dir = "/Users/joregan/Playing/hsi/audio/whisperx-json/"

In [35]:
import requests
import json
from pathlib import Path

headers = {
    "Authorization": f"Token {api_token}"
}

In [11]:
def get_projects():
    req = requests.get(f"{host}projects", headers=headers)
    assert req.status_code == 200
    data = json.loads(req.text)
    return data

In [21]:
def get_project_id_from_name(name):
    projects = get_projects()
    for res in projects["results"]:
        if res["title"].strip() == name.strip():
            return res["id"]

In [131]:
get_project_id_from_name("Inter rest")

4

In [31]:
def get_tasks(projectid):
    req = requests.get(f"{host}tasks", headers=headers, params={"project": projectid})
    assert req.status_code == 200
    data = json.loads(req.text)
    return data

In [45]:
def index_task_filestem_to_id(tasks_data):
    tasks = tasks_data["tasks"]
    mapping = {}
    for task in tasks:
        task_id = task["id"]
        if "storage_filename" in task:
            task_raw_path = task["storage_filename"]
        else:
            task_raw_path = task["data"]["audio"]
        if not task_raw_path:
            continue
        task_stem = task_raw_path.split("/")[-1]
        mapping[task_stem] = task_id
    return mapping

In [22]:
get_project_id_from_name("Speaker 3")

1

In [132]:
tasks = get_tasks(4)

In [133]:
mapping = index_task_filestem_to_id(tasks)

In [134]:
mapping

{'hsi_4_0717_209_001_inter.wav': 23,
 'hsi_4_0717_209_002_inter.wav': 24,
 'hsi_4_0717_209_003_inter.wav': 25,
 'hsi_4_0717_210_001_inter.wav': 26,
 'hsi_4_0717_210_003_inter.wav': 27,
 'hsi_4_0717_211_001_inter.wav': 28,
 'hsi_4_0717_211_002_inter.wav': 29,
 'hsi_4_0717_211_003_inter.wav': 30,
 'hsi_4_0717_222_002_inter.wav': 31,
 'hsi_4_0717_222_003_inter.wav': 32,
 'hsi_4_0717_227_002_inter.wav': 33,
 'hsi_4_0717_227_003_inter.wav': 34,
 'hsi_4_0717_227_004_inter.wav': 35,
 'hsi_5_0718_210_001_inter.wav': 36,
 'hsi_5_0718_210_002_inter.wav': 37,
 'hsi_5_0718_210_003_inter.wav': 38,
 'hsi_5_0718_211_002_inter.wav': 39,
 'hsi_5_0718_211_003_inter.wav': 40,
 'hsi_5_0718_222_002_inter.wav': 41,
 'hsi_5_0718_222_003_inter.wav': 42,
 'hsi_5_0718_227_001_inter.wav': 43,
 'hsi_5_0718_227_002_inter.wav': 44,
 'hsi_6_0718_209_002_inter.wav': 45,
 'hsi_6_0718_209_003_inter.wav': 46,
 'hsi_6_0718_210_001_inter.wav': 47,
 'hsi_6_0718_210_002_inter.wav': 48,
 'hsi_6_0718_211_001_inter.wav': 49,
 

ID: 71

In [97]:
import json

def convert_json(filename):
    with open(filename) as inf:
        data = json.load(inf)

    outputs = []
    for segment in data["segments"]:
        start = segment["start"]
        end = segment["end"]
        text = segment["text"]

        segment = {
            "value": {
                "start": start,
                "end": end,
                "channel": 0,
                "labels": ["Speech"]
            },
            "from_name": "labels",
            "to_name": "audio",
            "type": "labels",
        }
        rec = {
            "value": {
                "start": start,
                "end": end,
                "channel": 0,
                "text": [text.strip()]
            },
            "from_name": "transcription",
            "to_name": "audio",
            "type": "textarea",
        }
        outputs.append(segment)
        outputs.append(rec)

    return outputs

In [96]:
def post_results(id, task, project, results):
    ep = f"{host}annotations/{id}/?taskID={task}&project={project}"

    cur_headers = {i: headers[i] for i in headers}
    cur_headers["Content-type"] = "application/json"

    content = {
        "was_cancelled": False,
        "ground_truth": False,
        "project": project,
        "draft_id": 0,
        "parent_prediction": None,
        "parent_annotation": None,
        "result": results
    }
    r = requests.patch(ep, data=json.dumps(content), headers=cur_headers)
    return r

In [129]:
file = f"{input_dir}hsi_3_0715_209_006_main.json"
data = convert_json(file)

In [130]:
r = post_results(98, 22, 1, data)
print(r.text)

{"id":98,"result":[{"value":{"start":9.994,"end":10.839,"channel":0,"labels":["Speech"]},"from_name":"labels","to_name":"audio","type":"labels"},{"value":{"start":9.994,"end":10.839,"channel":0,"text":["Comprehensible."]},"from_name":"transcription","to_name":"audio","type":"textarea"},{"value":{"start":57.443,"end":58.064,"channel":0,"labels":["Speech"]},"from_name":"labels","to_name":"audio","type":"labels"},{"value":{"start":57.443,"end":58.064,"channel":0,"text":["Yeah, further."]},"from_name":"transcription","to_name":"audio","type":"textarea"},{"value":{"start":58.264,"end":58.444,"channel":0,"labels":["Speech"]},"from_name":"labels","to_name":"audio","type":"labels"},{"value":{"start":58.264,"end":58.444,"channel":0,"text":["Yeah."]},"from_name":"transcription","to_name":"audio","type":"textarea"},{"value":{"start":73.675,"end":74.576,"channel":0,"labels":["Speech"]},"from_name":"labels","to_name":"audio","type":"labels"},{"value":{"start":73.675,"end":74.576,"channel":0,"text":

In [135]:
count = 99
for task in mapping:
    jsonfile = task.replace(".wav", ".json")
    file = f"{input_dir}{jsonfile}"
    data = convert_json(file)
    r = post_results(count, mapping[task], 4, data)
    count += 1
    print(r.text)

{"id":99,"result":[{"value":{"start":4.246,"end":6.347,"channel":0,"labels":["Speech"]},"from_name":"labels","to_name":"audio","type":"labels"},{"value":{"start":4.246,"end":6.347,"channel":0,"text":["Okay, I started recording, I think that should be good."]},"from_name":"transcription","to_name":"audio","type":"textarea"},{"value":{"start":10.37,"end":17.494,"channel":0,"labels":["Speech"]},"from_name":"labels","to_name":"audio","type":"labels"},{"value":{"start":10.37,"end":17.494,"channel":0,"text":["Okay, yeah, I think I will just move in the room and then, yeah, we can... Yeah."]},"from_name":"transcription","to_name":"audio","type":"textarea"},{"value":{"start":19.635,"end":22.197,"channel":0,"labels":["Speech"]},"from_name":"labels","to_name":"audio","type":"labels"},{"value":{"start":19.635,"end":22.197,"channel":0,"text":["Let me know if I'm standing in a... Is it good here?"]},"from_name":"transcription","to_name":"audio","type":"textarea"},{"value":{"start":23.297,"end":23.4